# Dependency Installation

In [1]:
!pip install musdb stempeg youtube-dl openunmix stempeg transformers pydub touchaudio pathlib kenlm pocketsphinx demucs==3.0.5
!pip install pandas scikit-learn speechrecognition google-cloud-speech
!pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

  Using cached youtube_dl-2021.12.17-py2.py3-none-any.whl.metadata (1.5 kB)
ERROR: Could not find a version that satisfies the requirement touchaudio (from versions: none)
ERROR: No matching distribution found for touchaudio
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch===1.4.0 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0

In [2]:
!pip install stempeg==0.1.6
!pip install musdb
!pip install pydub
!pip pathlib
!pip sklearn
!pip install openunmix
import torch
import torchaudio
import numpy as np
import scipy
import stempeg
import os
from google.colab import files
from IPython.display import Audio, display
import musdb
from google.colab import files
import re
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import pydub
from pydub import AudioSegment
import torchaudio
from pathlib import Path
from openunmix import predict
import pandas as pd
import speech_recognition as sr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from pydub import AudioSegment
from google.cloud import speech, storage

print("test")

  Using cached stempeg-0.2.3-py3-none-any.whl.metadata (9.0 kB)
Using cached stempeg-0.2.3-py3-none-any.whl (963 kB)
  Attempting uninstall: stempeg
    Found existing installation: stempeg 0.1.6
    Uninstalling stempeg-0.1.6:
      Successfully uninstalled stempeg-0.1.6
ERROR: unknown command "pathlib"
ERROR: unknown command "sklearn" - maybe you meant "search"
test


# Vocal Isolation

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# uploaded = files.upload()

# input_wav = 'idgaf.wav'
input_wav = 'idgaf.wav'
uploaded = {Path(input_wav).name: open(input_wav, 'rb')}

In [4]:
start = 0

audio, rate = stempeg.read_stems(
    list(uploaded.keys())[0],
    sample_rate=44100,
    start=start,
    duration=None
)

estimates = predict.separate(
    audio=torch.as_tensor(audio).float(),
    rate=44100,
    device=device,
)

vocals = estimates.get('vocals', None)
bass = estimates.get('bass', None)
other = estimates.get('other', None)
drums = estimates.get('drums', None)



Downloading: "https://zenodo.org/records/5069601/files/vocals-bccbd9aa.pth" to /root/.cache/torch/hub/checkpoints/vocals-bccbd9aa.pth
100%|██████████| 108M/108M [00:07<00:00, 15.3MB/s]
Downloading: "https://zenodo.org/records/5069601/files/drums-69e0ebd4.pth" to /root/.cache/torch/hub/checkpoints/drums-69e0ebd4.pth
100%|██████████| 108M/108M [00:06<00:00, 16.4MB/s]
Downloading: "https://zenodo.org/records/5069601/files/bass-2ca1ce51.pth" to /root/.cache/torch/hub/checkpoints/bass-2ca1ce51.pth
100%|██████████| 108M/108M [01:02<00:00, 1.80MB/s]
Downloading: "https://zenodo.org/records/5069601/files/other-c8c5b3e6.pth" to /root/.cache/torch/hub/checkpoints/other-c8c5b3e6.pth
100%|██████████| 108M/108M [00:07<00:00, 15.2MB/s]


In [5]:
if vocals is not None:
    vocals_np = vocals.detach().cpu().numpy()[0]
    drums_np = drums.detach().cpu().numpy()[0]
    bass_np = bass.detach().cpu().numpy()[0]
    other_np = other.detach().cpu().numpy()[0]


    vocals_tensor = torch.tensor(vocals_np)
    drums_tensor = torch.tensor(drums_np)
    bass_tensor = torch.tensor(bass_np)
    other_tensor = torch.tensor(other_np)

    if vocals_tensor.ndim == 1:
        vocals_tensor = vocals_tensor.unsqueeze(0)

    display(Audio(vocals_tensor.numpy(), rate=rate))


    output_path = input_wav[:-4] + "IsolatedVocals.wav"

    print(f"Isolated vocals saved to {output_path}")
else:
    print("Vocals track not found in estimates.")

In [ ]:

if vocals is not None:
    vocals_np = vocals.detach().cpu().numpy()[0]

    vocals_tensor = torch.tensor(vocals_np)

    if vocals_tensor.ndim == 1:
        vocals_tensor = vocals_tensor.unsqueeze(0)


    display(Audio(vocals_tensor.numpy(), rate=rate))

    output_path = input_wav[:-4] + "IsolatedVocals.wav"
    torchaudio.save(output_path, vocals_tensor, rate)

    if drums is not None:
        drums_np = drums.detach().cpu().numpy()[0]
        drums_tensor = torch.tensor(drums_np)
        if drums_tensor.ndim == 1:
            drums_tensor = drums_tensor.unsqueeze(0)
        torchaudio.save(input_wav[:-4] + "IsolatedDrums.wav", drums_tensor, rate)

    if bass is not None:
        bass_np = bass.detach().cpu().numpy()[0]
        bass_tensor = torch.tensor(bass_np)
        if bass_tensor.ndim == 1:
            bass_tensor = bass_tensor.unsqueeze(0)
        torchaudio.save(input_wav[:-4] + "IsolatedBass.wav", bass_tensor, rate)

    if other is not None:
        other_np = other.detach().cpu().numpy()[0]
        other_tensor = torch.tensor(other_np)
        if other_tensor.ndim == 1:
            other_tensor = other_tensor.unsqueeze(0)
        torchaudio.save(input_wav[:-4] + "IsolatedOther.wav", other_tensor, rate)

    print(f"Isolated stems saved to {output_path}")


# Profanity Censorship

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'redacted'

# List of profane words
profane_words = [
    'shit', 'shitty', 'fuck', 'fucking', 'fuckin', 'fuk', 'fuc', 'fukin', 'fuckin', 'fucked', 'fucker', 'bitch',
    'bitches', 'cunt', 'dick', 'dicks', 'dickhead', 'dumbass', 'asshole', 'ass', 'asses', 'bastard',
    'damn', 'damned', 'crap', 'crappy', 'pussy', 'pussies', 'cock', 'cocks', 'dildo',
    'dildos', 'nigger', 'nigga', 'niggas', 'niggers', 'niggaz', 'nigas', 'nigaz','faggot', 'fag', 'fags', 'slut', 'sluts', 'whore',
    'whores', 'bastards', 'stupidass', 'bollocks', 'bugger', 'buggers', 'wanker', 'wankers', 'twat',
    'twats', 'dumbass', 'dumbasses', 'motherfucker', 'motherfuckers', 'faggots', 'tits', 'tit', 'fuking',
    'arsehole', 'arseholes', 'bollock', 'bollocks', 'bollocking', 'son of a bitch', 'sons of bitches', 'dumass', 'dumasses', 'dumbasses', 'retard', 'retards',
    'sex'
]

BUCKET_NAME = 'music-cleaner'

def preprocess_audio(audio_file):
    song = AudioSegment.from_file(audio_file, format='wav')

    song = song.set_channels(1)

    song = song.apply_gain(-song.max_dBFS)

    trimmed_song = song
    preprocessed_audio_file = 'preprocessed_audio.wav'
    trimmed_song.export(preprocessed_audio_file, format='wav')

    return preprocessed_audio_file

def upload_to_gcs(file_path, bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob_name = os.path.basename(file_path)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    return f'gs://{bucket_name}/{blob_name}'

def long_running_recognize(gcs_uri):
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US',
        enable_word_time_offsets=True,
        model='latest_long',
        use_enhanced=True
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    print('Waiting for operation to complete...')
    response = operation.result(timeout=300)

    return response

def get_profanity_timestamps(response, profane_words):
    profane_timestamps = []

    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word.lower()
            if word in profane_words:
                start_time = word_info.start_time.total_seconds() * 1000
                end_time = word_info.end_time.total_seconds() * 1000
                profane_timestamps.append((int(start_time), int(end_time)))

    return profane_timestamps

def mute_profanity(audio_file, timestamps):

    song = AudioSegment.from_wav(audio_file)
    for start, end in timestamps:
        song = song[:start] + AudioSegment.silent(duration=end-start) + song[end:]
    return song

def process_song(audio_file):
    preprocessed_audio_file = preprocess_audio(audio_file)
    gcs_uri = upload_to_gcs(preprocessed_audio_file, BUCKET_NAME)
    response = long_running_recognize(gcs_uri)

    profane_timestamps = get_profanity_timestamps(response, profane_words)
    if not profane_timestamps:
        print("No profanity detected")
        return None, None, None

    recognized_lyrics = " ".join([result.alternatives[0].transcript for result in response.results])
    print(f"Recognized Lyrics: {recognized_lyrics}")

    cleaned_song = mute_profanity(preprocessed_audio_file, profane_timestamps)

    cleaned_song_path = audio_file + 'Clean.wav'
    cleaned_song.export(cleaned_song_path, format='wav')
    return cleaned_song_path, recognized_lyrics, profane_timestamps

In [ ]:
display(Audio("idgafIsolatedVocals.wav"))

In [9]:
from pydub import AudioSegment

# Load the cleaned vocals

cleaned_vocals_path = 'idgafIsolatedVocals.wavClean.wav'
cleaned_vocals = AudioSegment.from_file(cleaned_vocals_path, format='wav')


drums_path = 'idgafIsolatedDrums.wav'
bass_path = 'idgafIsolatedBass.wav'
other_path = 'idgafIsolatedOther.wav'

drums = AudioSegment.from_file(drums_path, format='wav')
bass = AudioSegment.from_file(bass_path, format='wav')
other = AudioSegment.from_file(other_path, format='wav')


combined = cleaned_vocals.overlay(drums)
combined = combined.overlay(bass)
combined = combined.overlay(other)


final_output_path = 'idgafFinalClean.wav'
combined.export(final_output_path, format='wav')

print(f"Final cleaned song with instrumentals saved to {final_output_path}")
display(Audio(final_output_path))

Final cleaned song with instrumentals saved to idgafFinalClean.wav


In [ ]:
display(Audio(final_output_path))

# Testing/Evaluation

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
import numpy as np
import mir_eval

def load_audio(file_path, target_sample_rate=None):
    """Load audio file, resample if needed, and return as a numpy array."""
    audio = AudioSegment.from_file(file_path, format='wav')
    if target_sample_rate is not None:
        audio = audio.set_frame_rate(target_sample_rate)
    samples = np.array(audio.get_array_of_samples())

    if audio.channels > 1:
        samples = samples.reshape((-1, audio.channels)).mean(axis=1)
    return samples, audio.frame_rate

def align_audio(vocals, instruments):
    """Align two audio arrays to the same length."""
    min_length = min(len(vocals), len(instruments))
    return vocals[:min_length], instruments[:min_length]

def normalize_audio(audio):
    """Normalize audio signal to have zero mean and unit variance."""
    audio = audio - np.mean(audio)
    audio = audio / np.std(audio)
    return audio

def evaluate_vocal_isolation(isolated_vocals, mixed_audio):
    """Calculate the Mean Squared Error between isolated vocals and mixed audio vocals."""
    mse = np.mean((isolated_vocals - mixed_audio) ** 2)
    print(f"Vocal Isolation Mean Squared Error: {mse:.4f}")
    return mse

def evaluate_audio_separation(reference, estimated):
    """Calculate SDR, SIR, and SAR using mir_eval."""
    sdr, sir, sar, _ = mir_eval.separation.bss_eval_sources(reference[np.newaxis, :], estimated[np.newaxis, :])
    print(f"SDR: {sdr[0]:.4f} dB, SIR: {sir[0]:.4f} dB, SAR: {sar[0]:.4f} dB")
    return sdr[0], sir[0], sar[0]

def plot_spectrogram(audio_samples, sample_rate, title):
    """Plot the spectrogram of an audio signal."""
    plt.figure(figsize=(12, 6))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_samples)), ref=np.max)
    librosa.display.specshow(D, sr=sample_rate, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

common_sample_rate = 44100

vocals_samples, vocals_rate = load_audio('mapsA.wav', target_sample_rate=common_sample_rate)
instruments_samples, instruments_rate = load_audio('mapsIsolatedVocals.wav', target_sample_rate=common_sample_rate)

assert vocals_rate == instruments_rate, "Sample rates of audio files must match!"
aligned_vocals, aligned_instruments = align_audio(vocals_samples, instruments_samples)
normalized_vocals = normalize_audio(aligned_vocals)
normalized_instruments = normalize_audio(aligned_instruments)
vocal_isolation_mse = evaluate_vocal_isolation(normalized_vocals, normalized_instruments)
sdr, sir, sar = evaluate_audio_separation(normalized_vocals, normalized_instruments)
plot_spectrogram(normalized_vocals, common_sample_rate, "Spectrogram of Studio Isolated Vocals- Maps")
plot_spectrogram(normalized_instruments, common_sample_rate, "Spectrogram of Our Isolated Vocals- Maps")


In [14]:
import os
from pydub import AudioSegment
from google.cloud import speech, storage
from difflib import SequenceMatcher

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'redacted'

profane_words = [
    'shit', 'shitty', 'fuck', 'fucking', 'fuckin', 'fuk', 'fuc', 'fukin', 'fuckin', 'fucked', 'fucker', 'bitch',
    'bitches', 'cunt', 'dick', 'dicks', 'dickhead', 'dumbass', 'asshole', 'ass', 'asses', 'bastard',
    'damn', 'damned', 'crap', 'crappy', 'pussy', 'pussies', 'cock', 'cocks', 'dildo',
    'dildos', 'nigger', 'nigga', 'niggas', 'niggers', 'niggaz', 'nigas', 'nigaz','faggot', 'fag', 'fags', 'slut', 'sluts', 'whore',
    'whores', 'bastards', 'stupidass', 'bollocks', 'bugger', 'buggers', 'wanker', 'wankers', 'twat',
    'twats', 'dumbass', 'dumbasses', 'motherfucker', 'motherfuckers', 'faggots', 'tits', 'tit', 'fuking',
    'arsehole', 'arseholes', 'bollock', 'bollocks', 'bollocking', 'son of a bitch', 'sons of bitches', 'dumass', 'dumasses', 'dumbasses', 'retard', 'retards',
    'sex'
]

BUCKET_NAME = 'music-cleaner'

actual_lyrics = """
I don't fuck with you
You little stupid ass bitch, I ain't fuckin' with you
You little, you little dumb ass bitch, I ain't fuckin' with you
I got a million trillion things I'd rather fuckin' do
Than to be fuckin' with you, little stupid ass

I don't give a fuck, I don't give a fuck
I don't, I don't, I don't give a fuck
Bitch, I don't give a fuck about you
Or anything that you do
Don't give a fuck about you, or anything that you do

I heard you got a new man, I see you takin' a pic
Then you post it up, thinkin' that it's makin' me sick
I see you callin', I be makin' it quick
I'ma answer that shit like, "I don't fuck with you."
Bitch, I got no feelings to go
I swear I had it up to here, I got no ceilings to go
I mean, for real, fuck how you feel
Fuck your two cents if it ain't goin' towards the bill
Yeah, and every day I wake up celebratin' shit
Why? ‘Cause I just dodged a bullet from a crazy bitch
I stuck to my guns, that's what made me rich
That's what put me on, that's what got me here
That's what made me this
And everything that I do is my first name (B-I-G)
These hoes chase bread, aw damn, she got a bird brain
Ain't nothin' but trill in me, aw man, silly me
I just bought a crib, three stories; that bitch a trilogy
And you know I'm rollin' weed that's fuckin' up the ozone
I got a bitch that text me she ain't got no clothes on
And then another one text, then your ass next
And I'ma text your ass back, like…

I don't fuck with you
You little stupid ass bitch, I ain't fuckin' with you
You little, you little dumb ass bitch, I ain't fuckin' with you
I got a million trillion things I'd rather fuckin' do
Than to be fuckin' with you, little stupid ass

I don't give a fuck, I don't give a fuck
I don't, I don't, I don't give a fuck
Bitch, I don't give a fuck about you
Or anything that you do
Don't give a fuck about you, or anything that you do

Got a million things on my mind, executive deals online
Limited amount of time, chasin' these dollar signs
And you ain't on your grind
You liable to find me up in the MGM casino in the D
Fuckin' off fetti I could've put on property
From the Bay to the Murder Mitten
My niggas put murder missions
She choosin', that's her decision, free my niggas in prison
On the phone with a bitch who can't do shit
For a pimp but make a nigga hella rich
Got a blunt in my dental, blowin' hemp in a rental
On my way to Sacramento, late night; Arsenio
I'm never sentimental, go hard or go homeless
Barely Harley, I'm chromeless, you might end up domeless
I bet you she into me, her cheddar, she givin' me
I'll make a bitch stand outside forever, like the Statue of Liberty
Rest in pimp, Pimp C, underground king of the South
I raise my styrofoam up, and pour some drink in my mouth
Why you always coming around with bad news?
Say you want me to win, but hope I lose
Askin' if I rock with other niggas in the crew
But them niggas cool, it's just that…

Bitch, I ain't fuckin' with you (Little biatch!)
You little stupid ass bitch, I ain't fuckin' with you (Nuh-uh!)
You little, you little dumb ass bitch, I ain't fuckin' with you
I got a million trillion things I'd rather fuckin' do
Than to be fuckin' with you, little stupid ass

I don't give a fuck, I don't give a fuck
I don't, I don't, I don't give a fuck
Bitch, I don't give a fuck about you
Or anything that you do
Don't give a fuck about you, or anything that you do
I don't give a fuck, I don't give a fuck
I don't, I don't, I don't give a fuck
Bitch, I don't give a fuck about you
Or anything that you do
Don't give a fuck about you, or anything that you do

I got a new chick that I gotta thank God for
I got a new whip that I gotta thank the lot for
Yeah, I got a lot but want a lot more
Yeah, we in the buildin', but I'm tryna take it to the top floor
I swear I hear some new bullshit every day I'm wakin' up
It seems like nowadays everybody breakin' up
That shit can break you down if you lose a good girl
I guess you need a bad bitch to come around and make it up
I guess drama makes for the best content
Everything got a bad side, even a conscience
Now you're drinkin' 'til you're unconscious
Feel me when you get a fine bitch
Just don't forget to read the fine print
Life got me meditatin' like I'm in the Himalayas
Keep it G with the L lit on me like the elevator
Yeah, I know that karma's too real
So I hope you doin' cool, but still…

Stupid ass bitch, I ain't fuckin' with you
Little stupid ass… I ain't fuckin with…
I ain't fuckin'… I ain't, I ain't fuckin' with you
I ain't fuckin' with you
"""

def preprocess_audio(audio_file):

    song = AudioSegment.from_file(audio_file, format='wav')
    song = song.set_channels(1) # to mono
    song = song.apply_gain(-song.max_dBFS)
    trimmed_song = song
    preprocessed_audio_file = 'preprocessed_audio.wav'
    trimmed_song.export(preprocessed_audio_file, format='wav')

    return preprocessed_audio_file

def upload_to_gcs(file_path, bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob_name = os.path.basename(file_path)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    return f'gs://{bucket_name}/{blob_name}'

def long_running_recognize(gcs_uri):
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US',
        enable_word_time_offsets=True,
        model='latest_long',
        use_enhanced=True
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=300)

    return response

def get_profanity_timestamps(response, profane_words):
    profane_timestamps = []

    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word.lower()
            if word in profane_words:
                start_time = word_info.start_time.total_seconds() * 1000
                end_time = word_info.end_time.total_seconds() * 1000
                profane_timestamps.append((int(start_time), int(end_time), word))

    return profane_timestamps

def mute_profanity(audio_file, timestamps):
    song = AudioSegment.from_wav(audio_file)
    for start, end, _ in timestamps:
        song = song[:start] + AudioSegment.silent(duration=end-start) + song[end:]
    return song

def process_song(audio_file):
    preprocessed_audio_file = preprocess_audio(audio_file)
    gcs_uri = upload_to_gcs(preprocessed_audio_file, BUCKET_NAME)

    response = long_running_recognize(gcs_uri)

    profane_timestamps = get_profanity_timestamps(response, profane_words)
    if not profane_timestamps:
        print("No profanity detected")
        return None, None, None

    recognized_lyrics = " ".join([result.alternatives[0].transcript for result in response.results])
    print(f"Recognized Lyrics: {recognized_lyrics}")

    cleaned_song = mute_profanity(preprocessed_audio_file, profane_timestamps)
    cleaned_song_path = audio_file + 'Clean.wav'
    cleaned_song.export(cleaned_song_path, format='wav')
    return cleaned_song_path, recognized_lyrics, profane_timestamps

def evaluate_profanity_detection(audio_file, actual_lyrics):
    cleaned_song_path, recognized_lyrics, detected_curse_words = process_song(audio_file)

    actual_words = actual_lyrics.lower().split()
    recognized_words = recognized_lyrics.lower().split()
    matcher = SequenceMatcher(None, actual_words, recognized_words)
    transcription_accuracy = matcher.ratio() * 100

    ground_truth_curse_words = [word for word in actual_words if word in profane_words]
    detected_words = [word for _, _, word in detected_curse_words]

    misclassified_curse_words = [word for word in ground_truth_curse_words if word not in detected_words]
    misclassified_non_curse_words = [word for word in detected_words if word not in ground_truth_curse_words]

    total_curse_words = len(ground_truth_curse_words)
    detected_curse_word_count = len(detected_words)
    percent_profanity_removed = (detected_curse_word_count / total_curse_words) * 100 if total_curse_words > 0 else 0

    # Output the evaluation results
    print(f"Actual Lyrics: {actual_lyrics}")
    print(f"Recognized Lyrics: {recognized_lyrics}")
    print(f"Transcription Accuracy: {transcription_accuracy:.2f}%")
    print(f"Ground Truth Curse Words: {ground_truth_curse_words}")
    print(f"Detected Curse Words: {detected_words}")
    print(f"Misclassified Curse Words: {misclassified_curse_words}")
    print(f"Misclassified Non-Curse Words: {misclassified_non_curse_words}")
    print(f"Percent Profanity Removed: {percent_profanity_removed:.2f}%")

    return {
        "actual_lyrics": actual_lyrics,
        "recognized_lyrics": recognized_lyrics,
        "transcription_accuracy": transcription_accuracy,
        "ground_truth_curse_words": ground_truth_curse_words,
        "detected_curse_words": detected_words,
        "misclassified_curse_words": misclassified_curse_words,
        "misclassified_non_curse_words": misclassified_non_curse_words,
        "percent_profanity_removed": percent_profanity_removed
    }

evaluation_results = evaluate_profanity_detection('idgafIsolatedVocals.wavClean.wav', actual_lyrics)


Waiting for operation to complete...
Recognized Lyrics:     uh  you  you  look you  I got a million things I read  in a bit came with you little stupid as I'll give I'll give I'll give it I don't give up about you or anything that you don't give up about you or anything that you do I heard you got a new man I see you taking the pick do you post it up thinking that it's making me sick but I see you calling I'll be making it quick I'm an incident like I fuck with you it's I got no feelings to go I swear I had it up to here I Got No Ceilings to go I mean for real okay and every day I wake up celebrated why cuz I just that's the bullet from a credit I stuck to my guns that's what made me Rich that's what put me on that's what got me here that's what made me this everything that I do is my first name he's host Chase bread okay she got a bird brain ain't nothing but really me I just bought a crew 3 stories it's a Trilogy and you know I'm rolling we just got no clothes on and then another one